In [23]:
import pandas as pd

df = pd.read_pickle('attackers_df.pkl')

# VIEWING OPTIONS IN THE NOTEBOOK
from sklearn import set_config; set_config(display='diagram')

In [2]:
df

,player_id,games_2022,minutes_played_2022,goals_2022,assists_2022,goals_against_2022,goals_for_2022,clean_sheet_2022,name,position,...,squad_size,current_club_domestic_competition_id,agent_name,contract_expiration_date,term_days_remaining,market_value_in_eur,highest_market_value_in_eur,value_goals_for_2022,yellow_cards_2022,red_cards_2022
21969,751084,11.0,367.0,0.0,0.0,25.0,5.0,0.0,Matvey Ivakhnov,Attack,...,25.0,RU1,NaN,2023-06-30,94,100000.0,100000.0,12.60,3.0,0.0
21970,79642,14.0,1175.0,1.0,1.0,14.0,17.0,5.0,Shinji Okazaki,Attack,...,24.0,BE1,PRO Profil GmbH,2023-06-30,94,250000.0,10000000.0,71.57,1.0,0.0
21971,217923,17.0,1357.0,1.0,1.0,29.0,14.0,3.0,Khyzyr Appaev,Attack,...,25.0,RU1,NaN,2023-06-30,94,300000.0,700000.0,35.28,3.0,0.0
21972,525692,20.0,944.0,3.0,2.0,37.0,16.0,3.0,Georgiy Gongadze,Attack,...,25.0,RU1,NaN,2024-06-30,460,500000.0,500000.0,40.32,3.0,0.0
21973,237623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Oleksandr Filippov,Attack,...,24.0,BE1,ProStar,2023-06-30,94,250000.0,1300000.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28467,342389,19.0,1510.0,4.0,2.0,28.0,24.0,4.0,Anders Dreyer,Attack,...,26.0,BE1,Elite Consulting,2027-06-30,1555,9000000.0,9000000.0,77.34,1.0,0.0
28468,423604,15.0,901.0,3.0,2.0,22.0,22.0,3.0,Francis Amuzu,Attack,...,26.0,BE1,Christophe Henrotay,2025-06-30,825,6500000.0,8000000.0,92.62,1.0,0.0
28469,24484,14.0,897.0,4.0,5.0,21.0,21.0,2.0,Lior Refaelov,Attack,...,26.0,BE1,Scoutpush by Dudu Dahan,2023-06-30,94,500000.0,4000000.0,88.41,2.0,0.0
28472,903611,1.0,2.0,0.0,0.0,1.0,0.0,0.0,Nilson Angulo,Attack,...,26.0,BE1,Kancha Agency,2027-06-30,1555,1300000.0,1500000.0,0.00,0.0,0.0


In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
#from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer

In [4]:
def encode(frame, feature):
    '''
    function that custom encode a categorical feature ordered by the most impactful 
    sub-category on the SalePrice
    '''
    target = 'market_value_in_eur'
    # create a temporary dataframe  
    ordering = pd.DataFrame()
    # create an index with the unique values of the selected feature
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    # create a spmean column with the mean value of SalePrice 
    # for each category of the selected feature, sorted by value
    ordering['spmean'] = frame[[feature, target]]\
                         .groupby(feature).mean()[target]
    ordering = ordering.sort_values('spmean')
    # create a column ordering with the number from 1 to the number of
    # categories for the selected feature
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    # extracting index and that column to a dictionary
    ordering = ordering['ordering'].to_dict()
    # encoding string categories into integers, keeping the order 
    # of impact of the cateory to the saleprice mean
    # putting them in a new column feature_E
    #for cat, o in ordering.items():
        #frame.loc[frame[feature] == cat, feature+'_E'] = o
    return ordering

In [5]:
df.columns

Index(['player_id', 'games_2022', 'minutes_played_2022', 'goals_2022',
       'assists_2022', 'goals_against_2022', 'goals_for_2022',
       'clean_sheet_2022', 'name', 'position', 'sub_position', 'last_season',
       'foot', 'height_in_cm', 'age', 'country_of_citizenship',
       'country_of_birth', 'current_club_name', 'club_value', 'squad_size',
       'current_club_domestic_competition_id', 'agent_name',
       'contract_expiration_date', 'term_days_remaining',
       'market_value_in_eur', 'highest_market_value_in_eur',
       'value_goals_for_2022', 'yellow_cards_2022', 'red_cards_2022'],
      dtype='object')

In [6]:
features_to_ordinal_encode = ['country_of_citizenship','current_club_name','current_club_domestic_competition_id','agent_name']

In [32]:
for feature in features_to_ordinal_encode:
  print(encode(df, feature).keys())

dict_keys(['Indonesia', 'Tajikistan', 'Faroe Islands', 'Moldova', 'Belarus', 'Montserrat', 'Uganda', 'Northern Ireland', 'Peru', 'Azerbaijan', 'Curacao', 'Costa Rica', 'Ireland', 'Iceland', 'Mozambique', 'Congo', 'Sierra Leone', 'Grenada', 'Finland', 'Panama', 'Guadeloupe', 'South Africa', 'Iraq', 'Chad', 'Tunisia', 'Angola', 'Venezuela', 'Greece', 'Australia', 'North Macedonia', 'Tanzania', 'Cape Verde', 'Slovakia', 'Scotland', 'Palestine', 'Luxembourg', 'Ukraine', 'Latvia', 'Jordan', 'Martinique', 'Comoros', 'Russia', 'Romania', 'Honduras', 'Turkey', 'Montenegro', 'Zimbabwe', 'Ecuador', 'Chile', 'Bosnia-Herzegovina', 'The Gambia', 'Denmark', 'Dominican Republic', 'Kosovo', 'Mali', 'Guinea', 'Mauritania', 'New Zealand', 'Guinea-Bissau', 'Uzbekistan', 'Armenia', 'Netherlands', 'Japan', 'Morocco', 'Gabon', 'Ghana', 'Slovenia', 'Togo', 'Philippines', 'Israel', 'Switzerland', 'Trinidad and Tobago', 'Kazakhstan', 'Austria', 'Suriname', 'Cameroon', 'Burkina Faso', 'Algeria', 'Paraguay', 'Be

In [8]:
def impute_foot(players_df):
    #replace both with left and imputes right no nulls
    players_df['foot'].replace('Both', 'Left', inplace=True)
    players_df['foot'].replace(np.nan, 'Right', inplace=True)
    players_df['foot'].fillna('Right')

dict

In [28]:
def create_preproc_ordinal():
    
    feat_ordinal_dict = {
    
        "country_of_citizenship": ['Indonesia', 'Tajikistan', 'Faroe Islands', 'Moldova', 'Belarus', 'Montserrat', 'Uganda', 'Northern Ireland', 'Peru', 'Azerbaijan', 'Curacao', 'Costa Rica', 'Ireland', 'Iceland', 'Mozambique', 'Congo', 'Sierra Leone', 'Grenada', 'Finland', 'Panama', 'Guadeloupe', 'South Africa', 'Iraq', 'Chad', 'Tunisia', 'Angola', 'Venezuela', 'Greece', 'Australia', 'North Macedonia', 'Tanzania', 'Cape Verde', 'Slovakia', 'Scotland', 'Palestine', 'Luxembourg', 'Ukraine', 'Latvia', 'Jordan', 'Martinique', 'Comoros', 'Russia', 'Romania', 'Honduras', 'Turkey', 'Montenegro', 'Zimbabwe', 'Ecuador', 'Chile', 'Bosnia-Herzegovina', 'The Gambia', 'Denmark', 'Dominican Republic', 'Kosovo', 'Mali', 'Guinea', 'Mauritania', 'New Zealand', 'Guinea-Bissau', 'Uzbekistan', 'Armenia', 'Netherlands', 'Japan', 'Morocco', 'Gabon', 'Ghana', 'Slovenia', 'Togo', 'Philippines', 'Israel', 'Switzerland', 'Trinidad and Tobago', 'Kazakhstan', 'Austria', 'Suriname', 'Cameroon', 'Burkina Faso', 'Algeria', 'Paraguay', 'Belgium', 'Iran', 'Benin', 'Senegal', 'DR Congo', 'Sweden', 'Serbia', 'Wales', 'Croatia', 'Jamaica', 'Spain', 'Albania', 'Nigeria', 'Zambia', 'Portugal', 'Uruguay', 'Italy', 'United States', "Cote d'Ivoire", 'Germany', 'Colombia', 'Poland', 'Argentina', 'France', 'Canada', 'Brazil', 'Norway', 'Mexico', 'Hungary', 'Georgia', 'Czech Republic', 'Korea, South', 'England', 'Egypt'],
        "current_club_name":['Fk Minaj', 'Pfk Lviv', 'Gaziantep Fk', 'Ingulets Petrove', 'Kryvbas Kryvyi Rig', 'Metal Kharkiv', 'Nk Veres Rivne', 'Metalist 1925 Kharkiv', 'Chornomorets Odessa', 'Motherwell Fc', 'Dundee United Fc', 'Pas Lamia 1964', 'Pas Giannina', 'Lyngby Bk', 'Kilmarnock Fc', 'Rukh Lviv', 'Fk Oleksandriya', 'Ac Horsens', 'Kolos Kovalivka', 'Asteras Tripolis', 'Aberdeen Fc', 'Ross County Fc', 'St Mirren Fc', 'Go Ahead Eagles Deventer', 'St Johnstone Fc', 'Livingston Fc', 'Viborg Ff', 'Fakel Voronezh', 'Fc Volendam', 'Volos Nps', 'Ionikos Nikeas', 'Ofi Kreta', 'Torpedo Moskau', 'Istanbulspor', 'Fc Emmen', 'Apo Levadiakos', 'Panetolikos Gfs', 'Odense Boldklub', 'Rfc Seraing', 'Zorya Lugansk', 'Portimonense Sc', 'Sbv Excelsior Rotterdam', 'Gd Chaves', 'Heart Of Midlothian Fc', 'Vorskla Poltava', 'Aarhus Gf', 'Rio Ave Fc', 'Sv Zulte Waregem', 'Vv St Truiden', 'Fk Nizhny Novgorod', 'Umraniyespor', 'Sparta Rotterdam', 'Aalborg Bk', 'Boavista Porto Fc', 'Silkeborg If', 'Randers Fc', 'Kv Oostende', 'Ural Ekaterinburg', 'Fc Pacos De Ferreira', 'Fc Vizela', 'Casa Pia Ac', 'Rkc Waalwijk', 'Kas Eupen', 'Fc Arouca', 'Sc Cambuur Leeuwarden', 'Nec Nijmegen', 'Atromitos Athen', 'Fk Orenburg', 'Fk Sochi', 'Kv Kortrijk', 'Cd Santa Clara', 'Cs Maritimo', 'Fk Khimki', 'Hibernian Fc', 'Ac Ajaccio', 'Gd Estoril Praia', 'Fc Famalicao', 'Fortuna Sittard', 'Kvc Westerlo', 'Akhmat Grozny', 'Giresunspor', 'Mke Ankaragucu', 'Kv Mechelen', 'Kayserispor', 'Brondby If', 'Vitoria Guimaraes Sc', 'Sivasspor', 'Fc Twente Enschede', 'Fatih Karagumruk', 'Vitesse Arnheim', 'Sc Heerenveen', 'Cercle Brugge', 'Hatayspor', 'Dynamo Kiew', 'Alanyaspor', 'Kasimpasa', 'Krylya Sovetov Samara', 'Fc Midtjylland', 'Aj Auxerre', 'Aris Thessaloniki', 'Fk Rostov', 'Fc Nordsjaelland', 'Gil Vicente Fc', 'Fc Utrecht', 'Adana Demirspor', 'Antalyaspor', 'Aek Athen', 'Oud Heverlee Leuven', 'Fc Schalke 04', 'Konyaspor', 'Paok Thessaloniki', 'Clermont Foot 63', 'Sco Angers', 'Standard Luttich', 'Fc Groningen', 'Vfl Bochum', 'Rsc Charleroi', 'Shakhtar Donetsk', 'Istanbul Basaksehir Fk', 'Sampdoria Genua', 'Hellas Verona', 'Fc Cadiz', 'Us Cremonese', '1 Fc Koln', 'Fc Toulouse', 'Sk Dnipro 1', 'Panathinaikos Athen', 'Royale Union Saint Gilloise', 'Kaa Gent', 'Krc Genk', 'Rc Strassburg Alsace', 'Fk Krasnodar', 'Stade Brest 29', 'Spezia Calcio', 'Fc Elche', 'Us Lecce', 'Vfb Stuttgart', 'Fc Empoli', 'Fc Kopenhagen', 'Lokomotiv Moskau', 'Hertha Bsc', 'Olympiakos Piraus', 'Real Valladolid', 'Es Troyes Ac', 'Royal Antwerpen Fc', 'Sv Werder Bremen', 'Fc Augsburg', 'Az Alkmaar', 'Rcd Mallorca', 'Stade Reims', 'Zska Moskau', 'Rsc Anderlecht', 'Spartak Moskau', 'Fc Lorient', '1 Fsv Mainz 05', 'Rayo Vallecano', 'Ud Almeria', 'Espanyol Barcelona', 'Ac Monza', '1 Fc Union Berlin', 'Feyenoord Rotterdam', 'Dinamo Moskau', 'Glasgow Rangers', 'Celtic Glasgow', 'Fc Girona', 'Montpellier Hsc', 'Brighton Amp Hove Albion', 'Rc Lens', 'Ca Osasuna', 'Celta Vigo', 'Fc Nantes', 'Udinese Calcio', 'Fc Bologna', 'Sc Braga', 'Trabzonspor', 'Us Salernitana 1919', 'Besiktas Istanbul', 'Tsg 1899 Hoffenheim', 'Zenit St Petersburg', 'Psv Eindhoven', 'Fenerbahce Istanbul', 'Fc Getafe', 'Fc Turin', 'Vfl Wolfsburg', 'Borussia Monchengladbach', 'Sc Freiburg', 'Fc Brugge', 'Fc Valencia', 'Galatasaray Istanbul', 'Olympique Lyon', 'Afc Bournemouth', 'Us Sassuolo', 'Lazio Rom', 'Sporting Lissabon', 'Nottingham Forest', 'Fc Porto', 'Olympique Marseille', 'Fc Sevilla', 'Ac Florenz', 'Ogc Nizza', 'Fc Fulham', 'As Monaco', 'Fc Southampton', 'Athletic Bilbao', 'Ajax Amsterdam', 'Losc Lille', 'Real Betis Sevilla', 'Leeds United', 'Real Sociedad San Sebastian', 'Atalanta Bergamo', 'Wolverhampton Wanderers', 'Fc Everton', 'Eintracht Frankfurt', 'Benfica Lissabon', 'Ac Mailand', 'As Rom', 'Fc Brentford', 'Crystal Palace', 'Borussia Dortmund', 'Fc Stade Rennes', 'Fc Villarreal', 'Juventus Turin', 'Newcastle United', 'Aston Villa', 'West Ham United', 'Leicester City', 'Bayer 04 Leverkusen', 'Rasenballsport Leipzig', 'Atletico Madrid', 'Ssc Neapel', 'Inter Mailand', 'Manchester United', 'Fc Chelsea', 'Real Madrid', 'Fc Bayern Munchen', 'Fc Arsenal', 'Fc Barcelona', 'Tottenham Hotspur', 'Fc Liverpool', 'Fc Paris Saint Germain', 'Manchester City'],
        "current_club_domestic_competition_id": ['UKR1', 'DK1', 'GR1', 'SC1', 'BE1', 'RU1', 'NL1', 'TR1', 'PO1', 'FR1', 'L1', 'IT1', 'ES1', 'GB1'],
    }
    
    feat_ordinal = sorted(feat_ordinal_dict.keys())
    feat_ordinal_values_sorted = [feat_ordinal_dict[i] for i in feat_ordinal]
    
    encoder_ordinal = OrdinalEncoder(
        categories=feat_ordinal_values_sorted,
        handle_unknown="use_encoded_value",
        unknown_value=-1  # Considers unknown values as worse than "missing"
    )

    preproc_ordinal = make_pipeline(
    #   SimpleImputer(strategy="constant", fill_value="missing"),
        encoder_ordinal,
        StandardScaler()
    )
    
    
    return preproc_ordinal, feat_ordinal



def create_preproc_numerical():
    return make_pipeline(
        KNNImputer(),
        MinMaxScaler()
    )

def create_preproc_nominal():
    return make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OneHotEncoder(handle_unknown="ignore")
    )

def create_preproc(X, percentile = 75):
    """Create an (unfitted) sklearn preprocessing pipeline well suited for the Tranfermarkt Kaggle dataset

    Args:
        X (pandas.DataFrame): Feature matrix
        percentile (float): Percentage of feature to keep after one hot encoding

    Returns:
        sklearn.pipeline.Pipeline: un-fitted preprocessor
    """
    
    features_to_ordinal_encode = ['country_of_citizenship','current_club_name','current_club_domestic_competition_id','agent_name']
    
    preproc_ordinal, feat_ordinal = create_preproc_ordinal()
    
    preproc_numerical = create_preproc_numerical()
    feat_numerical = sorted(X.select_dtypes(
        include=["int64", "float64"]).columns)
    
    preproc_nominal = create_preproc_nominal()
    feat_nominal = sorted(
        list(set(X.columns) - set(feat_numerical) - set(feat_ordinal)))
    
    feature_transformer = ColumnTransformer(
        [
        ("numerical_encoder", preproc_numerical, feat_numerical),
        ("ordinal_encoder", preproc_ordinal, feat_ordinal),
        ("nominal_encoder", preproc_nominal, feat_nominal)
        ],
        remainder="drop")

    preproc = make_pipeline(
        feature_transformer
    )
    return preproc


In [29]:
preproc = create_preproc (df)

In [30]:
preproc

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical_encoder',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['age', 'assists_2022',
                                                   'clean_sheet_2022',
                                                   'club_value', 'games_2022',
                                                   'goals_2022',
                                                   'goals_against_2022',
                                                   'goals_for_2022',
                                                   'height_in_cm',
                                                   'highest_market_value_in_eur',
                                                   'last_season',
                                                   'mark...
                                                                   StandardScaler())]),
                                                  ['agent_name',
                                                   'country_of_citizenship',
                                                   'current_club_domestic_competition_id',
                                                   'current_club_name']),
                                                 ('nominal_encoder',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['contract_expiration_date',
                                                   'country_of_birth', 'foot',
                                                   'name', 'position',
                                                   'sub_position'])]))])

In [31]:
preproc.fit_transform(df)

TypeError: cannot pickle 'dict_keys' object